<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- Calculation of Actuals
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- [Creation of Ifrs Variables](../Import/5ImportScope-ToIfrsVar)
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the calculation of actuals values.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "2ImportScope-PresentValue"

# Actual Values

We consider 3 types of Actual values, which are distinguished through their [Estimate Type](../DataModel/DataStructure#estimate-type):
- [Actuals](#actuals) (A)
- [Advance Actuals](#advance-actuals) (AA)
- [Overdue Actuals](#overdue-actuals) (OA)

with the Estimate Type's system name shown between parenthesis above.

## Actuals
The Actuals correspond to WrittenActuals values and have estimate type $A$.
The only valid AoC Step is Cashflow (CF) with Novelty C. 
For the current period (no projection) the actual value corresponds to the input provided. If the actual input is not provided, a payment pattern input is used instead. Such payment pattern input is provided in the cashflow inputs with EstimateType PCE, AocType CL, and Novelty C.

$$
\text{Written Actual} (\text{AocStep}) = \left\{
\begin{array}{rl}
0 ~, & \text{if AoC Step is not CF,C} \\
\text{Actual Projection} ~, & \text{if ProjectionPeriod > 0 } \\
\text{Actual From Payment Pattern PCE} ~, & \text{if Imported Actual value is 0} \\
\text{Imported Actual} ~, & \text{otherwise}
\end{array}
\right.
$$

Where $\text{Actual Projection}$ corresponds to 
$$
\text{Actual Projection} (\text{CF, C}) = \left\{
\begin{array}{rl}
\text{Actual From Payment Pattern PCE - projection -} ~, & \text{if Payment Pattern PCE is available} \\
\Sigma_{\text{Novelty}} \text{PresentValue}(\text{CF}, \text{Novelty}) ~, & \text{otherwise}
\end{array}
\right.
$$

In [0]:
public interface WrittenActual : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<WrittenActual>(s => s
            .WithApplicability<ActualEmptyValue>(x => !(x.Identity.Id.AocType == AocTypes.CF && x.Identity.Id.Novelty == Novelties.C))
            .WithApplicability<ActualProjection>(x => !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode) && x.Identity.Id.ProjectionPeriod > 0)
            .WithApplicability<ActualFromPaymentPattern>(x => !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode) && Math.Abs(x.Value) < Precision));

    double Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear, Identity.Id.ProjectionPeriod);
}

public interface ActualEmptyValue : WrittenActual{
    double WrittenActual.Value => default;
}

public interface ActualProjection : WrittenActual{
    double WrittenActual.Value => GetStorage().GetValues(Identity.Id with {AocType = AocTypes.CL, Novelty = Novelties.C}, Identity.AmountType, EstimateTypes.PCE, Identity.AccidentYear).Any()
        ? GetScope<ActualFromPaymentPattern>(Identity).Value
        : GetStorage().GetNovelties(Identity.Id.AocType, StructureType.AocPresentValue)
            .Sum(novelty => GetScope<PresentValue>((Identity.Id with {AocType = AocTypes.CF, Novelty = novelty}, Identity.AmountType, EstimateTypes.BE, Identity.AccidentYear), o => o.WithContext(EconomicBases.C)).Value);
}

public interface ActualFromPaymentPattern : WrittenActual, IWithGetValueFromValues{
    double WrittenActual.Value => GetValueFromValues(
        GetStorage().GetValues(Identity.Id with {AocType = AocTypes.CL, Novelty = Novelties.C}, Identity.AmountType, EstimateTypes.PCE, Identity.AccidentYear),
        ValuationPeriod.Delta.ToString());
}

In [0]:
public interface Actual : IScope<ImportIdentity, ImportStorage>{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.A;
       
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] Actuals => 
        GetScope<ValidAmountType>(Identity.DataNode).AllImportedAmountTypes
            .SelectMany(amountType => GetStorage().GetAccidentYears(Identity.DataNode, Identity.ProjectionPeriod)
                .Select(accidentYear => (amountType, EstimateType, accidentYear, GetScope<WrittenActual>((Identity, amountType, EstimateType, accidentYear)).Value ))).ToArray();
}


## Accrual Actuals

The Accrual Actual family includes Advance Actual and Overdue Actual (actuals that are paid outside the period).

The following simplified AoC Chain applies for Advance and Overdue Actuals:
<br>&emsp;BOP : it corresponds to the opening value,
<br>&emsp;Cashflow : it corresponds to the movement in the period, 
<br>&emsp;Write-off : it corresponds to a write-off movement in the period,
<br>&emsp;EOP : it corresponds to the value at the end of the period.

The AoC Chain for Accruals is defined in input through the definition of the [AocStructure](../DataModel/DataStructure#aoc-configuration). Check [StructureType](../Constants/Enums#structure-type) for more options.

The Accrual Actual sets values for advance, and overdue as follows:

$$
\text{Accrual Actual} (\text{AoC step}) = \left\{
\begin{array}{rl}
0 ~, & \text{if AoC Step does not belong to the corresponding StructureType (AocAccrual)} \\
\text{Accrual Actual}(\rm{BOP}) + \text{Accrual Actual}(\rm{CF}) + \text{Accrual Actual}(\rm{WO}) ~, & \text{if AoC Step's Aoc Type = EOP} \\
0 ~, & \text{if Projection Period is beyond the first year projection} \\
\text{Accrual Actual}(\rm{EOP, p = 0}) ~, & \text{if Projection Period is the first year projection and AoC Step's Aoc Type is BOP} \\
- \text{Accrual Actual}(\rm{EOP, p = 0}), & \text{if Projection Period is the first year projection and AoC Step's Aoc Type is CF} \\
0, & \text{if Projection Period is the first year projection and AoC Step's Aoc Type is WO} \\
\text{Accrual Actual}(\rm{AoC~ Step, p = 0}), & \text{if Projection Period > 0} \\
\end{array}
\right.
$$

where p corresponds to the projection period and p = 0 identifies the current reporting period. 

The projection calculation for accruals reported above can also be summarized as follows: 
- quarterly projections within the current reporting year correspond to the current reporting period figures, 
- projection of the first future year end releases all accrual actuals through a cashflow (CF) step - this makes the projected Financial Performance insensitive to this release. 
- projection of the following periods do not compute any projected accrual.

In [0]:
public interface AccrualActual : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<AccrualActual>(s => s
            .WithApplicability<AccrualEmptyValues>(x => !x.GetStorage().GetAllAocSteps(StructureType.AocAccrual).Contains(x.Identity.Id.AocStep))
            .WithApplicability<AccrualEndOfPeriod>(x => !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode) && x.Identity.Id.AocType == AocTypes.EOP)
            .WithApplicability<AccrualEmptyValues>(x => !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode) && x.Identity.Id.ProjectionPeriod > x.GetStorage().FirstNextYearProjection) // projections beyond ReportingYear +1
            .WithApplicability<AccrualProjectionFirstYear>(x => !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode) && x.Identity.Id.ProjectionPeriod == x.GetStorage().FirstNextYearProjection && // projections ReportingYear +1
                                                                (x.Identity.Id.AocType == AocTypes.BOP || x.Identity.Id.AocType == AocTypes.CF))
            .WithApplicability<AccrualEmptyValues>(x => x.Identity.Id.ProjectionPeriod == x.GetStorage().FirstNextYearProjection && x.Identity.Id.AocType == AocTypes.WO)                                                                
            .WithApplicability<AccrualProjectionWithinFirstYear>(x => !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode) && x.Identity.Id.ProjectionPeriod > 0)
            );

    public double Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear, Identity.Id.ProjectionPeriod); 
}

public interface AccrualProjectionWithinFirstYear : AccrualActual{
    double AccrualActual.Value => GetScope<AccrualActual>((Identity.Id with {ProjectionPeriod = 0}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear )).Value;
}

public interface AccrualProjectionFirstYear : AccrualActual{
    private double signMultiplier => Identity.Id.AocType == AocTypes.BOP ? 1d : -1d;
    double AccrualActual.Value => signMultiplier * GetScope<AccrualActual>((Identity.Id with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.Id.ProjectionPeriod - 1}, 
        Identity.AmountType, Identity.EstimateType, Identity.AccidentYear)).Value;
}

public interface AccrualEndOfPeriod : AccrualActual{
    double AccrualActual.Value => GetScope<PreviousAocSteps>((Identity.Id, StructureType.AocAccrual)).Values.Sum(aocStep => 
        GetScope<AccrualActual>((Identity.Id with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear)).Value);
}

public interface AccrualEmptyValues : AccrualActual{
    double AccrualActual.Value => default;
}

## Advance Actuals

Advance Actuals are cash flows with due date inside the reporting period but occured *before* the reporting period - They include:
- Receivable Claims
- Receivable Expenses
- Payable Premiums

Advance Actuals are given by

$$
\text{Advance Actual} (\text{AoC step}) = \text{Accrual Actual} (\text{AoC step})|_{\text{Estimate Type} = AA}
$$

In [0]:
public interface AdvanceActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.AA;
    
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] Actuals => 
        GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
            .SelectMany(amountType => GetStorage().GetAccidentYears(Identity.DataNode, Identity.ProjectionPeriod)
                .Select(accidentYear => (amountType, EstimateType, accidentYear, GetScope<AccrualActual>((Identity, amountType, EstimateType, accidentYear)).Value) )).ToArray();
}

## Overdue Actuals

Overdue Actuals are cash flows with due date inside the reporting period but occur *after* the reporting period - They contain:
- Payable Claims
- Payable Expenses
- Receivable Premiums

The sign convention is the inverse of the default behavior - In particular: Premiums have positive value, whereas Claims and Expenses have negative value.

The Overdue Actuals are given by

$$
\text{Overdue Actual} (\text{AoC step}) = \text{Accrual Actual} (\text{AoC step})|_{\text{Estimate Type} = OA}
$$

In [0]:
public interface OverdueActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.OA;

    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] Actuals => 
        GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
            .SelectMany(amountType => GetStorage().GetAccidentYears(Identity.DataNode, Identity.ProjectionPeriod)
                .Select(accidentYear => (amountType, EstimateType, accidentYear, GetScope<AccrualActual>((Identity, amountType, EstimateType, accidentYear)).Value) )).ToArray();
}

# Deferrable

Deferral variables are computed here using cashflow inputs. We distinguish between two approaches depending on the EconomicBasisDriver defined in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters):
- L (Locked-in) or C (Current) : discounting is applied (cf. [Deferrable as cumulated discounted cash flow](#deferrable-as-cumulated-discounted-cash-flow)),
- N (Nominal) : discounting is not applied (cf. [Deferrable undiscounted](#deferrable-undiscounted)).

Amortization Factor for Deferral is computed using the discounting described above. <br> 
In addition, the amortization factor is computed using a pattern provided in the cash flow input with Amount Type DAE and AoC Type CL. If such a pattern is not found the calculation automatically defaults use the Coverage Units pattern (provided with Amount Type CU).

In [0]:
public interface DiscountedAmortizationFactorForDeferrals : IScope<ImportIdentity, ImportStorage>
{
    private string EconomicBasis => GetContext();
    double Value => GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType == AmountTypes.DAE
        ? GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, 0), o => o.WithContext(EconomicBasis)).Value
        : GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).Value;
    string EffectiveAmountType => GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType == AmountTypes.DAE
        ? AmountTypes.DAE
        : GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType;
}

## Deferrable as cumulated discounted cash flow
The cumulated discounted cash flow (CDC) or Present Value of deferral amount types is considered within this approach. 
The AoC Chain is the same we use for Technical Margin and is definedin input through the definition of the [AocStructure](../DataModel/DataStructure#aoc-configuration). Check [StructureType](../Constants/Enums#structure-type) for more options.
With respect to the Present Value AoC Chain, an Experience Adjustment and an Amortization step are computed. They represent the variance with respect to the actual received and to the release computed using a release pattern defined in input (in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters) or as cash flow), respectively. 

$$
\text{Deferrable} (\text{AoC step}) = \left\{
\begin{array}{rl}
\text{Deferrable}~(\text{BOP}) ~, & \text{if AoC Step's AoC Type = BOP, Novelty = I, and p is a quarterly projection in the current year } \\
\text{Deferrable}~(\text{BOP}, p = p - 1 ) ~, & \text{if AoC Step's AoC Type = BOP, Novelty = I, and p is a future year projection } \\
IAF \cdot~\sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = IA } \\
0 ~, & \text{if AoC Step's AoC Type = CF } \\
\text{PV}(\text{CF})- \text{Actual}(\text{CF})|_{\text{AmountType = DAE}} ~, & \text{if AoC Step's AoC Type = EA } \\
- \text{Amortization Factor} \cdot \sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step})  ~, & \text{if AoC Step's AoC Type = AM } \\
\sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = EOP } \\
\text{PV}(\text{AoC Step}) ~, & \text{ otherwise } \\
\end{array}
\right.
$$

In [0]:
public interface DiscountedDeferrable : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<DiscountedDeferrable>(s => s
            .WithApplicability<DeferrableWithIfrsVariable>(x => x.GetStorage().IsSecondaryScope(x.Identity.DataNode))
            .WithApplicability<DeferrableForBopProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
            .WithApplicability<DeferrableForBop>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I)
            .WithApplicability<DeferrableForIaStandard>(x => x.Identity.AocType == AocTypes.IA) // && x.Identity.Novelty == Novelties.I)
            //WithApplicability<DeferrableForIaNewBusiness>(x => x.Identity.AocType == AocTypes.IA)
            .WithApplicability<DeferrableDefaultValue>(x => x.Identity.AocType == AocTypes.CF)
            .WithApplicability<DeferrableEa>(x => x.Identity.AocType == AocTypes.EA)
            .WithApplicability<DeferrableAm>(x => x.Identity.AocType == AocTypes.AM)
            .WithApplicability<DeferrableEop>(x => x.Identity.AocType == AocTypes.EOP)
        );
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))] string EstimateType => EstimateTypes.DA;
    [IdentityProperty][NotVisible][Dimension(typeof(AmountType))] string AmountType => null;
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))] string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    
    double Value => GetStorage().GetDeferrableExpenses().Sum(at => 
        GetScope<PresentValue>((Identity, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value);
}

public interface DeferrableWithIfrsVariable : DiscountedDeferrable {
    double DiscountedDeferrable.Value => GetStorage().GetValue(Identity, AmountType, EstimateType, EconomicBasis, (int?)null, Identity.ProjectionPeriod);
}

public interface DeferrableForBopProjection : DiscountedDeferrable {
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<DiscountedDeferrable>(s => s
            .WithApplicability<DeferrableAfterFirstYear>(x => x.Identity.ProjectionPeriod >= x.GetStorage().FirstNextYearProjection));
    double DiscountedDeferrable.Value => GetScope<DiscountedDeferrable>(Identity with {ProjectionPeriod = 0}).Value;
}

public interface DeferrableAfterFirstYear : DiscountedDeferrable {
    double DiscountedDeferrable.Value => GetScope<DiscountedDeferrable>(Identity with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.ProjectionPeriod - 1}).Value;
}   

public interface DeferrableForBop : DiscountedDeferrable {
    //loop over amountTypes within deferrals to get bops
    double DiscountedDeferrable.Value => GetStorage().GetValue(Identity, null, EstimateTypes.DA, (int?)null, Identity.ProjectionPeriod);
}

public interface DeferrableForIaStandard : DiscountedDeferrable, InterestAccretionFactor {
    private double aggregatedValue => GetScope<PreviousAocSteps>((Identity, StructureTypes.AocTechnicalMargin)).Values
        .Sum(aoc => GetScope<DiscountedDeferrable>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);   
    double DiscountedDeferrable.Value => aggregatedValue * GetInterestAccretionFactor(EconomicBasis);
}

// public interface DeferrableForIaNewBusiness : DiscountedDeferrable, NewBusinessInterestAccretion {
//     private double[] nominalCashflows => GetStorage().GetDeferrableExpenses().Select(at => 
//         GetScope<NominalCashflow>((Identity, at, EstimateTypes.BE, (int?)null)).Values).AggregateDoubleArray();

//     double DiscountedDeferrable.Value => GetStorage().ImportFormat != ImportFormats.Cashflow || GetStorage().IsSecondaryScope(Identity.DataNode) // This is normally an applicability for the scope, but this is the only case --> to be re-checked
//         ? GetStorage().GetValue(Identity, null, EstimateType, EconomicBasis, (int?)null, Identity.ProjectionPeriod)
//         : -1d * GetInterestAccretion(nominalCashflows, EconomicBasis);
// }

public interface DeferrableDefaultValue : DiscountedDeferrable {
    double DiscountedDeferrable.Value => default;
}

public interface DeferrableEa : DiscountedDeferrable {
    private string referenceAocType => GetScope<ReferenceAocStep>(Identity).Values.First().AocType;
    double DiscountedDeferrable.Value => GetStorage().GetDeferrableExpenses().Sum(at =>
        GetStorage().GetNovelties(referenceAocType, StructureTypes.AocPresentValue)
        .Sum(n => GetScope<PresentValue>((Identity with {AocType = referenceAocType, Novelty = n}, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) -
             GetScope<WrittenActual>((Identity with {AocType = referenceAocType, Novelty = Novelties.C}, at, EstimateTypes.A, (int?)null)).Value);
}

public interface DeferrableAm : DiscountedDeferrable {
    private double amortizationFactor => GetScope<DiscountedAmortizationFactorForDeferrals>(Identity, o => o.WithContext(EconomicBasis)).Value;
    private double aggregatedValue => GetScope<PreviousAocSteps>((Identity, StructureTypes.AocTechnicalMargin)).Values
                                            .Sum(aocStep => GetScope<DiscountedDeferrable>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
    double DiscountedDeferrable.Value => Math.Abs(aggregatedValue) > Precision ? -1d * aggregatedValue * amortizationFactor : default;
}

public interface DeferrableEop : DiscountedDeferrable {
    double DiscountedDeferrable.Value => GetScope<PreviousAocSteps>((Identity, StructureTypes.AocTechnicalMargin)).Values
                                        .Sum(aocStep => GetScope<DiscountedDeferrable>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
}

## Deferrable undiscounted
The nominal cash flow of deferral amount types is considered within this approach. 
As Present Values are not used here, this methodology allows to amortize and release expenses occurring in different months within the reporting period. 
This is achieved by applying a shift to the release pattern. For this reason, this approach is to be used with a release pattern defined in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters).

The AoC Chain is simplified and considers: 
- BOP (I as opening value to be entered as cash flow, N for new expenses)
- AM,
- EOP.


$$
\text{Deferrable} (\text{AoC step}) = \left\{
\begin{array}{rl}

\text{Previous Period EOP} ~, & \text{if AoC Step's AoC Type = BOP } \\
- \text{Amortization Factor}(shift) \cdot \sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step})  ~, & \text{if AoC Step's AoC Type = AM } \\
\sum_{\text{Previous AoC Step}} \text{Deferrable}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = EOP } \\
0 ~, & \text{ otherwise } \\
\end{array}
\right.
$$

where the $shift$ corresponds to the month at which the cost occurs (starting from 0).



In [0]:
//TODO : 
// EstimateType from DA to DAC
// BOP,I only through Opening. 
public interface NominalDeferrable : IScope<(ImportIdentity Id, int MonthlyShift), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<NominalDeferrable>(s => s
                .WithApplicability<NominalDeferrableWithIfrsVariable>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow || x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode))
                .WithApplicability<BoPDeferrableProjection>(x => x.Identity.Id.AocType == AocTypes.BOP && x.Identity.Id.Novelty == Novelties.I && x.Identity.Id.ProjectionPeriod > 0)
                .WithApplicability<BoPDeferrable>(x => x.Identity.Id.AocType == AocTypes.BOP)
                .WithApplicability<AmDeferrable>(x => x.Identity.Id.AocType == AocTypes.AM)
                .WithApplicability<EopDeferrable>(x => x.Identity.Id.AocType == AocTypes.EOP)
            );
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))] string EstimateType => EstimateTypes.DA;
    [IdentityProperty][NotVisible][Dimension(typeof(AmountType))] string AmountType => null;
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))] string EconomicBasis => EconomicBases.N;
    double Value => default;
}

public interface NominalDeferrableWithIfrsVariable : NominalDeferrable {
    double NominalDeferrable.Value => GetStorage().GetValue(Identity.Id, AmountType, EstimateType, EconomicBasis, Identity.MonthlyShift, Identity.Id.ProjectionPeriod);
}

public interface BoPDeferrableProjection : NominalDeferrable{
    double NominalDeferrable.Value => GetScope<NominalDeferrable>((Identity.Id with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.Id.ProjectionPeriod - 1}, Identity.MonthlyShift)).Value;
}

public interface BoPDeferrable : NominalDeferrable{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<NominalDeferrable>(s => s.WithApplicability<NominalDeferrableFromIfrsVariable>(x => x.Identity.Id.Novelty == Novelties.I));
    private int projectionShift => GetStorage().GetShift(Identity.Id.ProjectionPeriod);
    double NominalDeferrable.Value => GetScope<NominalCashflow>((Identity.Id, AmountTypes.DAE, EstimateTypes.BE, (int?)null)).Values //loop over AM under DE
        .Skip(projectionShift + Identity.MonthlyShift).FirstOrDefault();
}

public interface NominalDeferrableFromIfrsVariable : NominalDeferrable{
    double NominalDeferrable.Value => GetStorage().GetValue(Identity.Id, AmountType, EstimateTypes.DA, EconomicBasis, Identity.MonthlyShift, Identity.Id.ProjectionPeriod);
}

public interface AmReferenceDeferrable: IScope<(ImportIdentity Id, int MonthlyShift), ImportStorage>{
    private int projectionShift => GetStorage().GetShift(Identity.Id.ProjectionPeriod);
    private IEnumerable<AocStep> previousAocSteps => GetScope<PreviousAocSteps>((Identity.Id, StructureTypes.AocTechnicalMargin)).Values.Where(aocStep => aocStep.Novelty != Novelties.C);
    double referenceCashflow => previousAocSteps
        .GroupBy(x => x.Novelty, (k, aocs) => aocs.Last())
        .Sum(aoc => GetScope<NominalCashflow>((Identity.Id with {AocType = aoc.AocType, Novelty = aoc.Novelty}, AmountTypes.DAE, EstimateTypes.BE, (int?)null)).Values
        .Skip(projectionShift + Identity.MonthlyShift).FirstOrDefault());
    //if no previous RawVariable, use IfrsVariable
    double Value => Math.Abs(referenceCashflow) >= Precision ? referenceCashflow : GetStorage().GetNovelties(AocTypes.BOP, StructureTypes.AocPresentValue).Sum(n => GetScope<NominalDeferrable>((Identity.Id with {AocType = AocTypes.BOP, Novelty = n}, Identity.MonthlyShift)).Value);
}

public interface AmDeferrable : NominalDeferrable{
    private IEnumerable<AocStep> referenceAocSteps => GetScope<ReferenceAocStep>(Identity.Id).Values; //Reference step of AM,C is CL,C
    private double referenceCashflow => referenceAocSteps.Sum(refAocStep => GetScope<AmReferenceDeferrable>((Identity.Id with {AocType = refAocStep.AocType, Novelty = refAocStep.Novelty}, Identity.MonthlyShift)).Value);

    double NominalDeferrable.Value => Math.Abs(referenceCashflow) > Precision ? -1d * referenceCashflow * GetScope<CurrentPeriodAmortizationFactor>((Identity.Id, AmountTypes.DAE, Identity.MonthlyShift), o => o.WithContext(EconomicBasis)).Value : default;
}

public interface EopDeferrable : NominalDeferrable{
    private IEnumerable<AocStep> previousAocSteps => GetScope<PreviousAocSteps>((Identity.Id, StructureTypes.AocTechnicalMargin)).Values;
    double NominalDeferrable.Value => previousAocSteps.Sum(aocStep => GetScope<NominalDeferrable>((Identity.Id with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, Identity.MonthlyShift)).Value);
}

# Premium Revenue

The discounting factors used in the computation of the Premium Revenue is driven by the EconomicBasisDriver defined in the [DataNodeParameter](../DataModel/DataStructure#data-node-parameters).

Similarly to the case of Deferral, the amortization factor for Premium Revenue is computed using a pattern provided in the cash flow input with Amount Type PR and AoC Type CL. If such a pattern is not found the calculation automatically defaults use the Coverage Units pattern (provided with Amount Type CU).

In [0]:
public interface DiscountedAmortizationFactorForRevenues : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))] private string EconomicBasis => GetContext();
    
    double Value => GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.PR, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType == AmountTypes.PR
        ? GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.PR, 0), o => o.WithContext(EconomicBasis)).Value
        : GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).Value;

    string EffectiveAmountType => GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.PR, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType == AmountTypes.PR
        ? AmountTypes.PR
        : GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(EconomicBasis)).EffectiveAmountType;
}

## Premium Revenue
The AoC Chain is the same as for [Deferrable](#deferrable).

$$
\text{Premium Revenue} (\text{AoC step}) = \left\{
\begin{array}{rl}
\text{Premium Revenue}~(\text{BOP}) ~, & \text{if AoC Step's AoC Type = BOP, Novelty = I, and p is a quarterly projection in the current year } \\
\text{Premium Revenue}~(\text{BOP}, p = p - 1 ) ~, & \text{if AoC Step's AoC Type = BOP, Novelty = I, and p is a future year projection } \\
IAF \cdot~ \sum_{\text{Previous AoC Step}} \text{Premium Revenue}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = IA } \\
0 ~, & \text{if AoC Step's AoC Type = CF } \\
\text{PV}(\text{CF})- \text{Actual}(\text{CF})|_{\text{Amount Type = Premiums}} ~, & \text{if AoC Step's AoC Type = EA } \\
- \text{Amortization Factor} \cdot \sum_{\text{Previous AoC Step}} \text{Premium Revenue}(\text{AoC Step})  ~, & \text{if AoC Step's AoC Type = AM } \\
\sum_{\text{Previous AoC Step}} \text{Premium Revenue}(\text{AoC Step}) ~, & \text{if AoC Step's AoC Type = EOP } \\
\text{PV}(\text{AoC Step}) ~, & \text{ otherwise } \\
\end{array}
\right.
$$

In [0]:
//only PAA LRC
public interface PremiumRevenue : IScope<ImportIdentity, ImportStorage>{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PremiumRevenue>(s => s
            .WithApplicability<PremiumRevenueWithIfrsVariable>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow || x.GetStorage().IsSecondaryScope(x.Identity.DataNode))
            .WithApplicability<PremiumRevenueForBopProjection>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I && x.Identity.ProjectionPeriod > 0)
            .WithApplicability<PremiumRevenueForBop>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I)
            .WithApplicability<PremiumRevenueForIaStandard>(x => x.Identity.AocType == AocTypes.IA) // && x.Identity.Novelty == Novelties.I)
            //.WithApplicability<PremiumRevenueForIaNewBusiness>(x => x.Identity.AocType == AocTypes.IA)
            .WithApplicability<PremiumRevenueDefaultValue>(x => new []{AocTypes.CF, AocTypes.EA}.Contains(x.Identity.AocType))
            //TODO compute EA but in the case of no LC EA is 0
            .WithApplicability<PremiumRevenueAm>(x => x.Identity.AocType == AocTypes.AM)
            .WithApplicability<PremiumRevenueEop>(x => x.Identity.AocType == AocTypes.EOP)
        );
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))] string EstimateType => EstimateTypes.R;
    [IdentityProperty][NotVisible][Dimension(typeof(AmountType))] string AmountType => null;
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))] string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    
    double Value => GetStorage().GetPremiums().Sum(at => 
        GetScope<PresentValue>((Identity, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value);
}

public interface PremiumRevenueWithIfrsVariable : PremiumRevenue {
    double PremiumRevenue.Value => GetStorage().GetValue(Identity, AmountType, EstimateType, EconomicBasis, (int?)null, Identity.ProjectionPeriod);
}

public interface PremiumRevenueForBopProjection : PremiumRevenue {
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PremiumRevenueForBopProjection>(s => s
            .WithApplicability<PremiumRevenueAfterFirstYear>(x => x.Identity.ProjectionPeriod >= x.GetStorage().FirstNextYearProjection));
    double PremiumRevenue.Value => GetScope<PremiumRevenue>(Identity with {ProjectionPeriod = 0}).Value;
}

public interface PremiumRevenueAfterFirstYear : PremiumRevenue {
    double PremiumRevenue.Value => GetScope<PremiumRevenue>(Identity with {AocType = AocTypes.EOP, Novelty = Novelties.C, ProjectionPeriod = Identity.ProjectionPeriod - 1}).Value;
}

public interface PremiumRevenueForBop : PremiumRevenue {
    double PremiumRevenue.Value => GetStorage().GetValue(Identity, AmountType, EstimateTypes.R, EconomicBasis, (int?)null, Identity.ProjectionPeriod);
}

public interface AggregatedPremiumRevenue : PremiumRevenue {
    double AggregatedValue => GetScope<PreviousAocSteps>((Identity, StructureTypes.AocTechnicalMargin)).Values
        .Sum(aoc => GetScope<PremiumRevenue>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
}

public interface PremiumRevenueForIaStandard : PremiumRevenue, InterestAccretionFactor {
    private double aggregatedValue => GetScope<AggregatedPremiumRevenue>(Identity).AggregatedValue;   
    double PremiumRevenue.Value => aggregatedValue * GetInterestAccretionFactor(EconomicBasis);
}

// public interface PremiumRevenueForIaNewBusiness : PremiumRevenue, NewBusinessInterestAccretion {
//     private double[] nominalCashflows => GetStorage().GetPremiums().Select(at => 
//         GetScope<NominalCashflow>((Identity, at, EstimateTypes.BE, (int?)null)).Values).AggregateDoubleArray();

//     double PremiumRevenue.Value => GetStorage().ImportFormat != ImportFormats.Cashflow || GetStorage().IsSecondaryScope(Identity.DataNode) // This is normally an applicability for the scope, but this is the only case --> to be re-checked
//         ? GetStorage().GetValue(Identity, null, EstimateType, EconomicBasis, (int?)null, Identity.ProjectionPeriod)
//         : -1d * GetInterestAccretion(nominalCashflows, EconomicBasis);
// }

public interface PremiumRevenueDefaultValue : PremiumRevenue {
    double PremiumRevenue.Value => default;
}

public interface PremiumRevenueAm : PremiumRevenue {
    private double AmortizationFactor => GetScope<DiscountedAmortizationFactorForRevenues>(Identity, o => o.WithContext(EconomicBasis)).Value;
    private double aggregatedValue => GetScope<AggregatedPremiumRevenue>(Identity).AggregatedValue;
    double PremiumRevenue.Value => Math.Abs(aggregatedValue) > Precision ? -1d * aggregatedValue * AmortizationFactor : default;
}

public interface PremiumRevenueEop : PremiumRevenue {
    double PremiumRevenue.Value => GetScope<AggregatedPremiumRevenue>(Identity).AggregatedValue;
}